In [1]:
import HSWN
import preprocess
import MF
import tagger
import sys
import xlrd
import translate
import moduleeng
import nltk
import algo
import sys
import xlsxwriter
from googletrans import Translator
from xlrd import open_workbook
translator = Translator(service_urls=['translate.google.co.in'])

In [4]:
reviews = []
book = open_workbook("hinglishText.xlsx")
sheet = book.sheet_by_index(0)
ctr = 0

for row in sheet.col(0):
    if ctr!=0:
        reviews.append(row.value.encode('utf-8'))
    ctr+=1
    
for i in range(0,10):
    str(reviews[i])
    
xbook = xlsxwriter.Workbook('Hinglish.xlsx')
xsheet = xbook.add_worksheet('vsTest')

for i in range(0,len(reviews)):
    translation = translator.translate(str(reviews[i]),src='en', dest='hi')
    xsheet.write(i,1,''.join((translation.text)))
    
posts = []

book = open_workbook('Hinglish.xlsx')
sheet = book.sheet_by_index(0)
ctr = 0
for row in sheet.col(1):
    if ctr!=0:
        #print(row.value)
        posts.append(row.value.encode('utf-8'))
    ctr+=1
    
#### Write posts with polirity
writeDoc = open('OUTPUT.csv','w+')
for i in range(len(posts)):
    posts[i] = preprocess.preProcess(posts[i])
    

sno = 1
for post in posts:
    #print(post)
## Get multipliers
    totalPol = 0.0

    tagdata = tagger.getTag(post)
    #print(tagdata)
    ## List of multiplying factors
    MFlist = MF.getMF(tagdata)

    for word in post.split(' '):
## Get tag info
        #print(MFlist)
        mult = -9999
        typ = -9999
        for mf in MFlist:
            #print(mf[0] == word)
            if mf[0] == word:
                mult = mf[1]
                typ = mf[2]

# If word exists in HSWN
        if HSWN.searchHSWN(word) != 'NF':
            wordPol = HSWN.searchHSWN(word)
            #print(wordPol)
                
####### HANDLE MULTIPLIER
            #print(mult)
            if mult > 0:
                wordPol = wordPol*mult
            else:
                pass

            totalPol += wordPol

    print("Review Number :", sno)
    writeDoc.write(str(sno)+',')
    
    if totalPol>0.1:
        writeDoc.write('1\n')
    elif totalPol<-0.1:
        writeDoc.write('-1\n')
    else:
        writeDoc.write('0\n')

    sno+=1
    
writeDoc.close()    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)